In [1]:
import csv
import requests
import re
import time
import itertools
import pandas as pd
import hgvs
import hgvs.parser
import hgvs.dataproviders.uta
import hgvs.assemblymapper

In [2]:
#Gets a protein sequence in FASTA format given the ensembl transcript ID
def get_protein_sequence(canonical_transcript):
    server = "http://grch37.rest.ensembl.org"
    ext = "/sequence/id/" + canonical_transcript + "?content-type=text/x-fasta;type=protein"
    r = requests.get(server + ext, headers={"Content-Type": "text/x-fasta"})
    #time.sleep(3)
    
    if not r.ok:
        try:
            r.raise_for_status()
            return "error"
        except requests.exceptions.HTTPError: #I should catch the error too or print, to find out which specific genes
            pass
    seqlist = r.text.split("\n", 1)
    
    if len(seqlist) == 2: #A lot of these transcripts are noncoding or introns, don't know why
        sequence = seqlist[1]
        sequence = sequence.replace("\n", "")
        return sequence
    #print("Sequence found")

#Adapted from https://github.com/xjenny2/phospho-programs/blob/master/ensembl.py

In [3]:
tsv_file = open("filtered_mapRef.tsv")
mart_tsv = csv.reader(tsv_file, delimiter="\t", quotechar='"')
df = pd.DataFrame(mart_tsv)

In [4]:
dfRef = df[df[6].astype(bool)] #filters tsv file such that it only considers all with a RefSeq Transcript

In [5]:
df2 = pd.read_csv('clinvar.csv', index_col = 0)
dfClinvar = pd.DataFrame(df2)

In [6]:
column = ['CHROM', 'POS', 'REF', 'ALT', 'MC', 'CLNSIG', 'NC', 'NM','NP','ENST', 'ProteinSeq', 'Distance to Next Methionine', 'Percentage Retained']
dfMap = pd.DataFrame(columns = column) #initializes dataframe with column headings

In [7]:
hdp = hgvs.dataproviders.uta.connect()
am = hgvs.assemblymapper.AssemblyMapper(hdp,assembly_name = 'GRCh38', alt_aln_method = "splign", replace_reference = True)

In [8]:
print(dfRef.shape)
print(dfClinvar.shape)

(15543, 7)
(31008, 10)


In [9]:
column = ['CHROM', 'POS', 'REF', 'ALT', 'MC', 'CLNSIG', 'NC', 'NM','NP','ENST', 'ProteinSeq', 'Distance to Next Methionine', 'Percentage Retained']
dfMap = pd.DataFrame(columns = column) #initializes dataframe with column headings
dfMap['ENST'] = dfMap['ENST'].fillna('0')

In [ ]:
data = [] #Fills in the dataframe with clinvar file information and converts gene to transcript to protein
transcript = ""
ensembl_id = ""
for index, row in dfClinvar.iterrows():
    if str(row[8]).startswith('NC') and row[8][-2] == '>':
        hgvs_g = str(row[8])
        hp = hgvs.parser.Parser()
        var_g = hp.parse_hgvs_variant(hgvs_g) #parses HGVS
        transcripts = am.relevant_transcripts(var_g)
        if transcripts[0].startswith('NR'):
            continue
        else:
            try:
                var_c = am.g_to_c(var_g, transcripts[0]) #converts Gene to Transcript
                var_p = am.c_to_p(var_c) #converts Transcript to Protein
                NM = str(var_c).split('.')[0] #obtains the initial transcript
                for x, y in dfRef.iterrows():
                    Match = y[6].split('.')[0] 
                    if NM == Match: #checks a map for a transcript
                        transcript = y[4]
                        ensembl_id = get_protein_sequence(transcript)
            except Exception:
                continue
    
    values = []
    values.append(row[0])
    values.append(row[1])
    values.append(row[3])
    values.append(row[4])
    values.append(row[7])
    values.append(row[9])
    values.append(row[8])
    values.append(str(var_c))
    values.append(str(var_p))
    values.append(transcript)
    values.append(ensembl_id)
    
    zipped = zip(column, values)
    dictionary = dict(zipped)
    data.append(dictionary)

Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_198576.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_018188.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=1520284&seq_stop=1520284&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_199454.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001291594.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googl

Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_024009.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001142604.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001852.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failur

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=99880678&seq_stop=99880678&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=99896323&seq_stop=99896323&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_033313.2&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_013296.4&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&

Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=152312743&seq_stop=152312743&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=152313385&seq_stop=152313385&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=152314107&seq_stop=152314107&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NC_000001.11&rettype=fasta&seq_start=154569572&seq_stop=154569572&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Fail

Failure 0/3; retry in 1 seconds
Failure 1/3; retry in 2 seconds
Failure 2/3; retry in 9 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_003005.3&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_001319173.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Failed to fetch https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&id=NM_000261.1&rettype=fasta&tool=bioutils&email=biocommons-dev@googlegroups.com
Failure 0/3; retry in 1 seconds
Normalization of intronic variants is not supported; returning unnormalized variant
Failed to fetch https://eutils.ncbi.nlm.nih.go

In [26]:
dfMap = dfMap.append(data, True) 

In [27]:
dfMap.shape

(1868, 13)

In [20]:
for index, row in dfMap.iterrows():
    Protein_Mutation_Position = int(re.findall(r'\d+', row[8])[-1])
    if row[10] != None:
        #Index and position are difference; Index = Position - 1
        sub_seq = row[10][Protein_Mutation_Position - 1:-1]
        distance = 0
        retained = 0
        for metIndex, aminoAcid in enumerate(sub_seq):
            if aminoAcid == 'M':
                distance = metIndex
                remain = sub_seq[metIndex:-1]
                retained = len(remain)/len(row[10])
                break
        dfMap.loc[index, 'Distance to Next Methionine'] = distance
        dfMap.loc[index, 'Percentage Retained'] = retained

In [22]:
dfMap.dropna()

,CHROM,POS,REF,ALT,MC,CLNSIG,NC,NM,NP,ENST,ProteinSeq,Distance to Next Methionine,Percentage Retained
0,1,943995,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.943995C>T,NM_152486.2:c.1888C>T,NP_689699.2:p.(Arg630Ter),,,0,0
1,1,944051,T,G,SO:0001587|nonsense,Uncertain_significance,NC_000001.11:g.944051T>G,NM_152486.2:c.1944T>G,NP_689699.2:p.(Tyr648Ter),,,0,0
4,1,1041582,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1041582C>T,NM_198576.3:c.1057C>T,NP_940978.2:p.(Gln353Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,17,0.818582
5,1,1042053,C,G,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1042053C>G,NM_198576.3:c.1275C>G,NP_940978.2:p.(Tyr425Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,154,0.716381
6,1,1045487,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1045487C>T,NM_198576.3:c.2500C>T,NP_940978.2:p.(Arg834Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,28,0.577995
7,1,1049672,C,T,SO:0001587|nonsense,Likely_pathogenic,NC_000001.11:g.1049672C>T,NM_198576.3:c.4621C>T,NP_940978.2:p.(Arg1541Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,121,0.186797
8,1,1232517,G,A,SO:0001587|nonsense,Likely_pathogenic,NC_000001.11:g.1232517G>A,NM_080605.3:c.239G>A,NP_542172.2:p.(Trp80Ter),ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...,59,0.574468
9,1,1232959,C,G,SO:0001587|nonsense,Likely_pathogenic,NC_000001.11:g.1232959C>G,NM_080605.3:c.681C>G,NP_542172.2:p.(Tyr227Ter),ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...,59,0.12766
10,1,1233041,C,T,SO:0001587|nonsense,Likely_pathogenic,NC_000001.11:g.1233041C>T,NM_080605.3:c.763C>T,NP_542172.2:p.(Gln255Ter),ENST00000379198,MKLLRRAWRRRAALGLGTLALCGAALLYLARCAAEPGDPRAMSGRS...,31,0.12766
11,1,1327311,G,T,SO:0001587|nonsense,Uncertain_significance,NC_000001.11:g.1327311G>T,NM_001029885.1:c.193G>T,NP_001025056.1:p.(Glu65Ter),ENST00000343938,MDDSETGFNLKVVLVSFKQCLDEKEEVLLDPYIASWKGLVRFLNSL...,17,0.61215


In [23]:
dfMap.head()

,CHROM,POS,REF,ALT,MC,CLNSIG,NC,NM,NP,ENST,ProteinSeq,Distance to Next Methionine,Percentage Retained
0,1,943995,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.943995C>T,NM_152486.2:c.1888C>T,NP_689699.2:p.(Arg630Ter),,,0,0
1,1,944051,T,G,SO:0001587|nonsense,Uncertain_significance,NC_000001.11:g.944051T>G,NM_152486.2:c.1944T>G,NP_689699.2:p.(Tyr648Ter),,,0,0
2,1,1014143,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014143C>T,NM_005101.3:c.163C>T,NP_005092.1:p.(Gln55Ter),ENST00000649529,None,NaN,NaN
3,1,1014359,G,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1014359G>T,NM_005101.3:c.379G>T,NP_005092.1:p.(Glu127Ter),ENST00000649529,None,NaN,NaN
4,1,1041582,C,T,SO:0001587|nonsense,Pathogenic,NC_000001.11:g.1041582C>T,NM_198576.3:c.1057C>T,NP_940978.2:p.(Gln353Ter),ENST00000379370,MAGRSHPGPLRPLLPLLVVAACVLPGAGGTCPERALERREEEANVV...,17,0.818582


In [24]:
dfMap.shape

(98, 13)